In [1]:
import pandas as pd 
import numpy as np
from scipy.optimize import curve_fit  
import matplotlib.pyplot as plt 
from datetime import datetime
from datetime import date
from  datetime import timedelta
import time
from impala.dbapi import connect
from impala.util import as_pandas
import json
import pymysql
from sqlalchemy import create_engine
import os

In [2]:
### 读取调整分割点final系数表
df_c = pd.read_excel('./分段系数表调整分割点v2.xlsx',sheet_name= '系数表',index_col = 'point')

In [3]:
def fund(x, a, b):  
    """gmv = a*exp(ln指数*b)"""
    return a * np.exp(b * x)

In [4]:
def fun_fit(x):
    '''按指数范围指数阶段函数'''
    global df_c
    list3 = df_c.index
    if  x<list3[1]:
        a = df_c.loc[list3[0],'a']
        b = df_c.loc[list3[0],'b']
    elif list3[1]<=x<list3[2]:
        a = df_c.loc[list3[1],'a']
        b = df_c.loc[list3[1],'b']  
    elif list3[2]<=x<list3[3]:
        a = df_c.loc[list3[2],'a']
        b = df_c.loc[list3[2],'b']  
    elif list3[3]<=x<list3[4]:
        a = df_c.loc[list3[3],'a']
        b = df_c.loc[list3[3],'b']
    elif list3[4]<=x<list3[5]:
        a = df_c.loc[list3[4],'a']
        b = df_c.loc[list3[4],'b']
    elif list3[5]<=x<list3[6]:
        a = df_c.loc[list3[5],'a']
        b = df_c.loc[list3[5],'b']  
    elif list3[6]<=x<list3[7]:
        a = df_c.loc[list3[6],'a']
        b = df_c.loc[list3[6],'b']  
    elif list3[7]<=x<list3[8]:
        a = df_c.loc[list3[7],'a']
        b = df_c.loc[list3[7],'b']
    elif list3[8]<=x<list3[9]:
        a = df_c.loc[list3[8],'a']
        b = df_c.loc[list3[8],'b']  
    elif list3[9]<=x<list3[10]:
        a = df_c.loc[list3[9],'a']
        b = df_c.loc[list3[9],'b']  
    elif list3[10]<=x<list3[11]:
        a = df_c.loc[list3[10],'a']
        b = df_c.loc[list3[10],'b']
    elif list3[11]<=x<list3[12]:
        a = df_c.loc[list3[11],'a']
        b = df_c.loc[list3[11],'b']
    elif list3[12]<=x<list3[13]:
        a = df_c.loc[list3[12],'a']
        b = df_c.loc[list3[12],'b']  
    elif list3[13]<=x<list3[14]:
        a = df_c.loc[list3[13],'a']
        b = df_c.loc[list3[13],'b']  
    elif list3[14]<=x<list3[15]:
        a = df_c.loc[list3[14],'a']
        b = df_c.loc[list3[14],'b']
    elif list3[15]<=x<list3[16]:
        a = df_c.loc[list3[15],'a']
        b = df_c.loc[list3[15],'b']  
    elif list3[16]<=x<list3[17]:
        a = df_c.loc[list3[16],'a']
        b = df_c.loc[list3[16],'b']  
    elif list3[17]<=x<list3[18]:
        a = df_c.loc[list3[17],'a']
        b = df_c.loc[list3[17],'b']
    elif list3[18]<=x<list3[19]:
        a = df_c.loc[list3[18],'a']
        b = df_c.loc[list3[18],'b']
    elif list3[19]<=x<list3[20]:
        a = df_c.loc[list3[19],'a']
        b = df_c.loc[list3[19],'b']
    elif list3[20]<=x<list3[21]:
        a = df_c.loc[list3[20],'a']
        b = df_c.loc[list3[20],'b']
    elif list3[21]<=x<list3[22]:
        a = df_c.loc[list3[21],'a']
        b = df_c.loc[list3[21],'b']
    else:
        a = df_c.loc[list3[22],'a']
        b = df_c.loc[list3[22],'b']
    y = fund(np.log(x),a,b)
    return y

In [5]:
def fitted(flag,col, data1):
    '''定义一个计算指数转换的函数
    '''
    col_list =col
    data2 = data1.copy()
    for i in col_list:
        data2[i.strip('_index')] = data2[i].map(lambda x: fun_fit(x))
        if flag == 1:
            data2[i+'_error_c']= (data2[i.strip('_index')]- data2[i+'_real'])/ data2[i+'_real']*100
    return data2

In [6]:
def format_round(data1=None, dict1 = None):
    """
    定义一个保留小数位数（四舍五入）的函数
    Args:
        data1-- A dataframe which has columns need to fromat the deciaml(default None.dataframe)
        dict1-- A dictory of colnames as key , numb of dicmal as value (default None, dict)
    Return:
        data2 the keep numb deciaml dataframe
    """
    col_list = dict1
    data2 = data1.copy()
    for key, value in col_list.items():
        data2[key] = data2[key].map(lambda x: np.round(x,value))
    return data2

In [8]:
#### 京东指数取数
# 连接集群hive/impala
def impala_connect(sql, **kwargs):
# impala 
    host = kwargs.get("host", 'impala.bjds.belle.lan')
    port = kwargs.get("port", 21051)    
    timeout = kwargs.get("timeout", 3600)
# hive
    # host = kwargs.get("host", 'impala.bjds.belle.lan')
    # port = kwargs.get("port", 10008)    
    # timeout = kwargs.get("timeout", 3600)
    user = kwargs.get("user", "lv.d.sz")
    password = kwargs.get("password", 'JHjLXpyQ')
    kerberos_service_name = kwargs.get("kerberos_service_name", "impala")
    conn = connect(host=host, port=port, timeout=timeout, user=user, password=password, kerberos_service_name=
                   kerberos_service_name,auth_mechanism='LDAP')
    cur = conn.cursor(user=user)
    if sql is not None:
        cur.execute(sql)
        try:
            df = as_pandas(cur)
        except:
            return cur
    return df

In [33]:
sql_vender_month = """
-- 二级品类店铺top100月度数据(2019年1月-)
select category, child_category, start_day, end_day, date_type, rank, shop_name, gmv_amount_index, order_num_index, visitor_index
from  bi_analysis.jd_sz_vender_ranks where 
category in ('时尚女鞋','流行男鞋','潮流女包','运动鞋包','运动服饰') and child_category = ""
-- and start_day >= '2020-03-01'
and length(date_type) = 6
order by start_day, category, rank
"""

In [34]:
df1 = impala_connect(sql_vender_month)

In [35]:
df1.loc[(df1.start_day == '2019-01-01') & (df1.category=='时尚女鞋')].shape

(100, 10)

In [37]:
fited_result = fitted(flag =0, col =['gmv_amount_index','order_num_index','visitor_index'],data1 = df1) # gmv 访客数 件单数 进行转换
df3 = format_round(fited_result,nub = 0, col =['order_num','visitor']) # 调整格式 访客数、件单数保留0位小数

In [49]:
# 重命名
def re_name_vender(df, platform, date_type, category3, norm_flag):
    df['platform'] = platform
    df['datetype']= date_type
    df.child_category = category3
    if norm_flag == 1:
        df.drop('date_type', axis =1)
    else:
        pass
    df.rename({'category':'category2_name','child_category':'category3_name','rank':'rank0'},axis=1,inplace= True)
    df= df[['platform','category2_name','category3_name','datetype','start_day','end_day','shop_name','rank0','gmv_amount_index','order_num_index','visitor_index','gmv_amount','order_num','visitor']]
    return df

In [50]:
df3 = re_name_vender(df3, platform = '京东',date_type = 'month',category3='all2',norm_flag =1)

In [51]:
df3.head()

,platform,category2_name,category3_name,datetype,start_day,end_day,shop_name,rank0,gmv_amount_index,order_num_index,visitor_index,gmv_amount,order_num,visitor
0,京东,时尚女鞋,all2,month,2019-01-01,2019-01-31,长龙鞋靴拼购店,1,65677140.0,68024.0,642107.0,8.382835e+06,1212.0,18119.0
1,京东,时尚女鞋,all2,month,2019-01-01,2019-01-31,宝丽芙旗舰店,2,44405456.0,695506.0,11179361.0,4.884425e+06,20046.0,748224.0
2,京东,时尚女鞋,all2,month,2019-01-01,2019-01-31,百丽集团官方旗舰店,3,41838576.0,372445.0,6093822.0,4.499678e+06,9206.0,332677.0
3,京东,时尚女鞋,all2,month,2019-01-01,2019-01-31,心刚拼购店,4,37366448.0,44239.0,594807.0,3.854197e+06,742.0,16470.0
4,京东,时尚女鞋,all2,month,2019-01-01,2019-01-31,足力健旗舰店,5,37073784.0,472459.0,3471572.0,3.813015e+06,12372.0,158415.0


In [ ]:
### 数据写入到mysql数据

In [52]:
conn = create_engine('mysql+mysqlconnector://sz_analy:Belle2020@sz@10.251.11.5:3306/bi_analysis_sz?charset=utf8',encoding ='utf-8')   
pd.io.sql.to_sql(df3,'jd_sz_vender_ranks_real', con=conn,chunksize=1000, schema='bi_analysis_sz',index=False,if_exists='replace') # append是追加的形式写入；replace

In [27]:
### 京东市场大盘[月度数据暂时不能使用 2级品类直接转换，需要3级品类相加]
# 取月度数据，二级，三级品类，
sql_industry = """
select category, child_category, start_day, end_day, date_type, cust_price_avg_value, ord_amt_index_value, convert_rate_value, uv_index_value,
ord_num_index_value, ord_pro_num_value from bi_analysis.jd_sz_industry_summary_data where category in ('运动服饰','流行男鞋','运动鞋包','潮流女包')  and length(date_type) = 6
"""
df_industry = impala_connect(sql_industry)

In [28]:
df_industry.head()

,category,child_category,start_day,end_day,date_type,cust_price_avg_value,ord_amt_index_value,convert_rate_value,uv_index_value,ord_num_index_value,ord_pro_num_value
0,流行男鞋,,2019-02-01,2019-02-28,201902,180.931580,1.766956e+09,0.0876,233715360.0,47214248.0,2394063.0
1,流行男鞋,商务休闲鞋,2019-01-01,2019-01-31,201901,227.374084,5.262763e+08,0.0720,65759856.0,9113962.0,225491.0
2,流行男鞋,鞋配件,2019-03-01,2019-03-31,201903,32.505112,1.233298e+08,0.2729,23559316.0,9068323.0,328341.0
3,流行男鞋,雨鞋/雨靴,2019-02-01,2019-02-28,201902,62.547394,3.126075e+07,0.1125,6867518.0,1241693.0,21201.0
4,潮流女包,卡包/零钱包,2019-08-01,2019-08-31,201908,27.717632,2.159004e+07,0.0567,15215511.0,1657828.0,41344.0


In [29]:
df_industry['category'].unique()

array(['流行男鞋', '潮流女包'], dtype=object)

In [11]:

df_industry.head()

,category,child_category,start_day,end_day,date_type,cust_price_avg_value,ord_amt_index_value,convert_rate_value,uv_index_value,ord_num_index_value,ord_pro_num_value
0,潮流女包,,2019-04-01,2019-04-30,201904,173.847824,854760320.0,0.0611,149737648.0,21243842.0,719425.0
1,潮流女包,钥匙包,2020-01-01,2020-01-31,202001,71.863686,5535181.0,0.0396,2393312.0,162000.0,1839.0
2,潮流女包,化妆包,2019-11-01,2019-11-30,201911,40.241833,9136534.0,0.0633,4361647.0,463001.0,5800.0
3,潮流女包,,2019-03-01,2019-03-31,201903,176.036102,970031104.0,0.0626,165656432.0,23960268.0,846711.0
4,潮流女包,手提包,2019-01-01,2019-01-31,201901,255.496674,186956000.0,0.0290,41434108.0,2686685.0,48313.0


In [12]:
# 重命名
def fill_category3(df):
    if df['child_category']=="":
        x='all2'
    else:
        x=df['child_category']
    return x
def re_name_industry(df, platform, date_type, norm_flag):
    df['platform'] = platform
    df['datetype']= date_type
    df['child_category'] = df.apply(lambda x: fill_category3(x), axis=1)
    if norm_flag == 1:
        df.drop('date_type', axis =1)
    else:
        pass
    df.rename({'category':'category2_name','child_category':'category3_name','cust_price_avg_value':'cust_price_value','ord_amt_index_value':'ord_gmv_index','uv_index_value':'uv_index','ord_num_index_value':'ord_num_index','ord_pro_num_value':'ord_pro_num_index'},axis=1,inplace= True)
    df= df[['platform','category2_name','category3_name','datetype','start_day','end_day','cust_price_value','ord_gmv_index','uv_index','ord_num_index','ord_pro_num_index']]
    return df

In [13]:
df_industry = re_name_industry(df=df_industry, platform='京东', date_type='month', norm_flag=1)

In [14]:
df_industry.head()

,platform,category2_name,category3_name,datetype,start_day,end_day,cust_price_value,ord_gmv_index,uv_index,ord_num_index,ord_pro_num_index
0,京东,潮流女包,all2,month,2019-04-01,2019-04-30,173.847824,854760320.0,149737648.0,21243842.0,719425.0
1,京东,潮流女包,钥匙包,month,2020-01-01,2020-01-31,71.863686,5535181.0,2393312.0,162000.0,1839.0
2,京东,潮流女包,化妆包,month,2019-11-01,2019-11-30,40.241833,9136534.0,4361647.0,463001.0,5800.0
3,京东,潮流女包,all2,month,2019-03-01,2019-03-31,176.036102,970031104.0,165656432.0,23960268.0,846711.0
4,京东,潮流女包,手提包,month,2019-01-01,2019-01-31,255.496674,186956000.0,41434108.0,2686685.0,48313.0


In [15]:
def fitted_indu(col, df):
    '''
    transfrom index to value for visitor_index, gmv_index, ord_pro_num_index...
    Args:
        col--A list of columns name of df which need to transtform to real value(list or str)
        df--A dataframe whcih include the columns of indexs (dataframe)
    Return:
        df2 --  the result of convert the index (dataframe)
    Note :
    行业大盘指数转换，月度2级品类暂时指数转换测算不太准，目前只对3级品类进行转换，由3级品类相加得到2级品类
    '''
    col_list =col
    data1 = df[df['category3_name']=='all2']
    for i in col_list:
        if i == 'ord_gmv_index':
            data1[i.strip('_index')]= data1[i].map(lambda x: 0)# 对于2级品类的数据较大，转换存在误差，故不对其进行转换，使用0 暂时填充替换
        else:
            data1[i.strip('_index')] = data1[i].map(lambda x: fun_fit(x))
    data2 = df[df['category3_name']!='all2']
    for i in col_list:
        data2[i.strip('_index')] = data2[i].map(lambda x: fun_fit(x))
    df2 = pd.concat([data1,data2])
    return df2

In [16]:
df_industry2 = fitted_indu(df = df_industry,col=['ord_gmv_index','uv_index','ord_num_index','ord_pro_num_index'])

In [17]:
def format_round(data1=None, dict1 = None):
    """
    定义一个保留小数位数（四舍五入）的函数
    Args:
        data1-- A dataframe which has columns need to fromat the deciaml(default None.dataframe)
        dict1-- A dictory of colnames as key , numb of dicmal as value (default None, dict)
    Return:
        data2 the keep numb deciaml dataframe
    """
    data2 = data1.copy()
    for key, value in dict1.items():
        n =value 
        data2[key] = data2[key].map(lambda x: np.round(x,n))
    return data2

In [18]:
df_industry2 = format_round(data1=df_industry2, dict1 ={'ord_gmv':2,'uv':0,'ord_num':0, 'ord_pro_num':0})

In [19]:
df_industry2.head()

,platform,category2_name,category3_name,datetype,start_day,end_day,cust_price_value,ord_gmv_index,uv_index,ord_num_index,ord_pro_num_index,ord_gmv,uv,ord_num,ord_pro_num
0,京东,潮流女包,all2,month,2019-04-01,2019-04-30,173.847824,8.547603e+08,149737648.0,21243842.0,719425.0,0.0,26387986.0,1781877.0,20922.0
3,京东,潮流女包,all2,month,2019-03-01,2019-03-31,176.036102,9.700311e+08,165656432.0,23960268.0,846711.0,0.0,30398846.0,2100259.0,25712.0
6,京东,流行男鞋,all2,month,2019-10-01,2019-10-31,179.544754,2.815011e+09,363418784.0,73957712.0,4177978.0,0.0,91967617.0,9880366.0,202017.0
39,京东,潮流女包,all2,month,2019-08-01,2019-08-31,170.936661,9.631475e+08,201468752.0,23458760.0,924657.0,0.0,39985278.0,2040432.0,28743.0
48,京东,流行男鞋,all2,month,2020-03-01,2020-03-31,164.655777,2.138210e+09,249916208.0,57218108.0,3058053.0,0.0,54123352.0,6926563.0,134150.0


In [22]:
df_industry2[df_industry2.category3_name!='all2'].head()

,platform,category2_name,category3_name,datetype,start_day,end_day,cust_price_value,ord_gmv_index,uv_index,ord_num_index,ord_pro_num_index,ord_gmv,uv,ord_num,ord_pro_num
1,京东,潮流女包,钥匙包,month,2020-01-01,2020-01-31,71.863686,5535181.0,2393312.0,162000.0,1839.0,2.929793e+05,97469.0,3353.0,29.0
2,京东,潮流女包,化妆包,month,2019-11-01,2019-11-30,40.241833,9136534.0,4361647.0,463001.0,5800.0,5.713257e+05,213836.0,12065.0,86.0
4,京东,潮流女包,手提包,month,2019-01-01,2019-01-31,255.496674,186956000.0,41434108.0,2686685.0,48313.0,3.601038e+07,4439848.0,113290.0,819.0
5,京东,潮流女包,斜挎包,month,2019-07-01,2019-07-31,74.237633,186562560.0,71779640.0,7175878.0,166663.0,3.590430e+07,9479885.0,413665.0,3469.0
7,京东,潮流女包,单肩包,month,2019-07-01,2019-07-31,180.867874,512560832.0,111770696.0,10885603.0,376828.0,1.496541e+08,17529552.0,722059.0,9341.0


In [24]:
conn= create_engine('mysql+mysqlconnector://sz_analy:Belle2020@sz@10.251.11.5:3306/bi_analysis_sz?charset=utf8',encoding ='utf-8')   
pd.io.sql.to_sql(df_industry2,'jd_sz_industry_summary_data_real', con=conn,chunksize=1000, schema='bi_analysis_sz',index=False,if_exists='append') # append是追加的形式写入；replace